In [1]:
### Function to create a 3D channel mesh and a 2D membrane mesh
###
### 


from dolfinx import mesh, fem, default_scalar_type, plot
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import functionspace
from dolfinx.mesh import locate_entities_boundary, exterior_facet_indices
from mpi4py import MPI
import ufl
import numpy as np
import pyvista
import dolfinx
import pyvista
import dolfinx.plot

#Parameters for 3D channel
nx, ny, nz = 6, 6, 6        # Mesh resolution in x, y, z
L, W, H = 8, 8, 3              # Channel dimensions: length, width, height


#Create 3D channel mesh
def create_mesh_3d_and_2d(L,W,H,nx,ny,nz):
    #L = length, W = width, H = height
    #nx, ny, nz = number of elements in x, y, z directions
    #This function creates a 3D mesh representing a rectangular channel
    #and a 2D mesh representing the membrane at the top surface of the channel
    #Returns both meshes
    mesh3d = dolfinx.mesh.create_box(MPI.COMM_WORLD, [[0, 0, 0], [L, W, H]], [nx, ny, nz])
    mesh2d = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD, [[0, 0], [L, W]], [nx, ny])
    return mesh3d, mesh2d

domain_3d, domain_2d = create_mesh_3d_and_2d(L, W, H, nx, ny, nz)

#Function to define constant pressure on the 2D membrane
def pressure_function(domain_2d, scalar_pressure):
    return fem.Constant(domain_2d, default_scalar_type(scalar_pressure))

#Define constant negative pressure applied to the membrane
pressure = pressure_function(domain_2d, -0.7)

def calculate_deformation(mesh_2d, pressure):
    #mesh_2d: 2D mesh representing the membrane
    #pressure: constant negative pressure applied to the membrane
    #returns the displacement solution of the membrane under pressure

    #Define scalar function space on 2D membrane
    V = functionspace(mesh_2d, ("Lagrange", 1))
    tdim = mesh_2d.topology.dim

    #Ensure boundary-facet-to-cell connectivity
    mesh_2d.topology.create_connectivity(tdim - 1, tdim)

    #Get coordinates of DoFs on 2D mesh
    dof_coords = V.tabulate_dof_coordinates().reshape((-1, 3))


    #Identify boundary DoFs — fix all DoFs outside the center region
    #We allow deformation only in central (membrane in middle of the top)
    outside_mask = (dof_coords[:, 0] <= 1/4*L) | (dof_coords[:, 0] >= 3/4*L) | \
                   (dof_coords[:, 1] <= 1/4*W) | (dof_coords[:, 1] >= 3/4*W)
    outside_dofs = np.where(outside_mask)[0].astype(np.int32)

    #Apply Dirichlet condition: fixed displacement (u = 0) on outer region
    bc_membrane = fem.dirichletbc(default_scalar_type(0), outside_dofs, V)

    #Define variational problem: -Δu = pressure
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)

    lhs = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    rhs = ufl.inner(pressure, v) * ufl.dx

   
    #Solve linear system with LU decomposition
    problem = LinearProblem(lhs, rhs, bcs=[bc_membrane],
        petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

    #Return the displacement solution
    return problem.solve()

#Calculate deformation of the 2D membrane under pressure
uh = calculate_deformation(domain_2d, pressure)

#Function to deform the 3D mesh based on the 2D membrane solution
def mesh_deformation(interface_values, fluid_domain):
    #interface_values: vertical displacements from 2D membrane
    #fluid_domain: original 3D fluid mesh to be deformed
    #returns the deformed 3D fluid mesh

    #Get function space and coordinates of 2D membrane solution
    V_2d = uh.function_space
    dof_coords_2d = V_2d.tabulate_dof_coordinates()
    #1D array of z-displacements on the 2D membrane
    values_2d = interface_values 

    #Create scalar function space on the 3D fluid mesh
    V = functionspace(fluid_domain, ("Lagrange", 1))
    tdim, fdim = fluid_domain.topology.dim, fluid_domain.topology.dim - 1
    fluid_domain.topology.create_connectivity(fdim, tdim)

    #Identify boundary facets
    all_boundary_facets = exterior_facet_indices(fluid_domain.topology)
    facet_top = locate_entities_boundary(fluid_domain, fdim,
        lambda x: np.isclose(x[2], fluid_domain.geometry.x[:, 2].max()))
    facet_rest = np.setdiff1d(all_boundary_facets, facet_top)

    #Get DoFs on top and other boundaries
    dofs_top = fem.locate_dofs_topological(V, fdim, facet_top)
    dofs_rest = fem.locate_dofs_topological(V, fdim, facet_rest)

    #Extract x-y coordinates of top DoFs
    dof_coords_3d = V.tabulate_dof_coordinates()[dofs_top]
    xy_coords_3d = dof_coords_3d[:, :2]

    #Interpolate the 2D deformation onto the 3D top surface
    from scipy.interpolate import griddata
    interpolated_values = griddata(
        dof_coords_2d[:, :2],   # Source: x-y points from 2D membrane
        values_2d,              # Corresponding z-displacements
        xy_coords_3d,           # Target: x-y on 3D top face
        method="linear",        # Linear interpolation
        fill_value=0.0          # Zero if outside 2D domain
    )

    #Create a function to hold the displacement values at top DoFs
    top_bc_function = fem.Function(V)
    top_bc_function.x.array[:] = 0.0
    for dof, val in zip(dofs_top, interpolated_values):
        top_bc_function.x.array[dof] = val
    top_bc_function.x.scatter_forward()

    #Dirichlet boundary conditions
    bc_rest = fem.dirichletbc(default_scalar_type(0), dofs_rest, V)      #Rest of the boundary: no displacement
    bc_top = fem.dirichletbc(top_bc_function, dofs_top)                  #Top: apply interpolated deformation

    #Laplace-type variational problem to get deformation
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
    L = fem.Constant(fluid_domain, default_scalar_type(0)) * v * ufl.dx

    #Solve PDE with the given boundary conditions
    problem = LinearProblem(a, L, bcs=[bc_top, bc_rest],
        petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    w = problem.solve()

    #Apply deformation in z-direction to the mesh geometry
    fluid_domain.geometry.x[:, 2] += w.x.array

    return fluid_domain



In [2]:
###Function to plot membrane deformation along the y-axis at a fixed x position
### 
### 


import numpy as np
import matplotlib.pyplot as plt
from dolfinx import geometry

def plot_membrane_deformation(domain_2d, domain_3d, pressure, calculate_deformation):
    """
    Evaluates and plots membrane deformation along the y-axis at a fixed x position.

    Parameters:
    - domain_2d: The 2D mesh/domain for membrane deformation.
    - domain_3d: The 3D mesh/domain for reference (for plotting limits).
    - pressure: Pressure input used for deformation calculation.
    - calculate_deformation: Function that computes deformation on the 2D domain given pressure.
    """
    # Small tolerance to avoid sampling exactly on domain edges (to prevent numerical issues)
    tol = 1e-4

    # Define a line of 101 points along the y-axis within the 2D domain bounds
    y = np.linspace(0 + tol, domain_2d.geometry.x[:, 1].max() - tol, 101)

    # Choose a fixed x position (middle of the domain in x-direction)
    x_val = 0.5 * domain_2d.geometry.x[:, 0].max()

    # Prepare 3D array of points for evaluation: (x, y, z=0) since 2D domain
    points = np.zeros((3, len(y)))
    points[0, :] = x_val    # constant x
    points[1, :] = y        # varying y
    # z remains zero

    # Build bounding box tree for spatial search in the 2D mesh
    bb_tree = geometry.bb_tree(domain_2d, domain_2d.topology.dim)

    # Find candidate cells that may contain each point
    cell_candidates = geometry.compute_collisions_points(bb_tree, points.T)

    # Confirm actual collisions to find the exact cells for each point
    colliding_cells = geometry.compute_colliding_cells(domain_2d, cell_candidates, points.T)

    # Extract the cell index for each point
    cells = []
    points_in_cells = []
    for i, point in enumerate(points.T):
        if len(colliding_cells.links(i)) > 0:
            points_in_cells.append(point)
            cells.append(colliding_cells.links(i)[0])

    points_in_cells = np.array(points_in_cells, dtype=np.float64)

    # Compute membrane deformation with provided function
    uh3 = calculate_deformation(domain_2d, pressure)

    # Evaluate deformation at each point in its corresponding cell
    u_values = uh3.eval(points_in_cells, cells)

    # Plot deformation along y-axis
    plt.plot([0, points_in_cells[:, 1].min()], [0, 0], color='blue')  # baseline before profile
    plt.plot(points_in_cells[:, 1], u_values, label='Deformation', color='blue')  # deformation profile
    plt.plot([points_in_cells[:, 1].max(), domain_3d.geometry.x[:, 1].max()], [0, 0], color='blue')  # baseline after profile

    plt.title('Membrane Deformation along the y-axis')
    plt.xlabel('y-coordinate')
    plt.ylabel('Deformation (u)')
    plt.grid(True)
    plt.legend()
    plt.show()



In [3]:
### Function to solve the Navier-Stokes equations on the deformed mesh
###
###



from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista
import dolfinx
from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter
from dolfinx.mesh import create_unit_square
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction,
                 div, dot, ds, dx, inner, lhs, nabla_grad, rhs, sym)

#Use previously defined mesh
#mesh = new_mesh1

# Time-stepping parameters
t = 0
T = 20
num_steps = 500
dt = T / num_steps

# Function to solve the Navier-Stokes equations
def solve_navier_stokes(mesh, t, T, num_steps, p_in, p_out):
    #mesh: 3D fluid mesh
    #t: initial time
    #T: final time
    #num_steps: number of time steps
    #p_in: inflow pressure
    #p_out: outflow pressure
    #returns:
    #glyphs: velocity glyphs for visualization
    #grid: VTK grid for visualization
    #V: function space for velocity
    #Q: function space for pressure
    #u_n: velocity function
    #p_n: pressure function

    dt = T / num_steps  #Time step size

    #Topological info for boundary detection
    tdim = mesh.topology.dim
    fdim = tdim - 1
    mesh.topology.create_connectivity(fdim, tdim)

    #Define function spaces: velocity (vector, degree 2) and pressure (scalar, degree 1)
    v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
    s_cg1 = element("Lagrange", mesh.topology.cell_name(), 1)
    V = functionspace(mesh, v_cg2)
    Q = functionspace(mesh, s_cg1)

    #Trial and test functions
    u = TrialFunction(V)
    v = TestFunction(V)
    p = TrialFunction(Q)
    q = TestFunction(Q)

    #Define boundary regions
    #Function to locate wall entities (bottom walls and side walls)
    def wall(x):
        return np.isclose(x[1], 0) | np.isclose(x[1], mesh.geometry.x[:, 1].max()) | \
           np.isclose(x[2], 0, atol=1e-4)

    #Locate wall entities and their DoFs
    wall_entities = locate_entities_boundary(mesh, fdim, wall)
    wall_dofs = fem.locate_dofs_topological(V, fdim, wall_entities)


    all_boundary_facets = dolfinx.mesh.exterior_facet_indices(mesh.topology)
    all_boundary_dofs = fem.locate_dofs_topological(V, fdim, all_boundary_facets)

    #Function to locate inflow entities (left wall)
    def inflow(x):
        return np.isclose(x[0], 0)

    #Locate inflow entities and their DoFs
    inflow_entities = locate_entities_boundary(mesh, fdim, inflow)
    inflow_dofs = locate_dofs_geometrical(Q, inflow)
    bc_inflow = dirichletbc(PETSc.ScalarType(p_in), inflow_dofs, Q)

    #Function to locate outflow entities (right wall)
    def outflow(x):
        return np.isclose(x[0], mesh.geometry.x[:, 0].max())

    #Locate outflow entities and their DoFs
    outflow_entities = locate_entities_boundary(mesh, fdim, outflow)
    outflow_dofs = locate_dofs_geometrical(Q, outflow)
    bc_outflow = dirichletbc(PETSc.ScalarType(p_out), outflow_dofs, Q)

    #Identify top wall facets (excluding inflow, outflow, and side walls)
    facets_top_wall = np.setdiff1d(all_boundary_facets, np.unique(np.concatenate((inflow_entities, outflow_entities, wall_entities))))

    #Locate DoFs on top wall facets
    top_wall_dofs = fem.locate_dofs_topological(V, fdim, facets_top_wall)

    #Define no-slip boundary condition for top wall
    u_noslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
    bc_noslip_top = dirichletbc(u_noslip, top_wall_dofs, V)

    #Define no-slip boundary condition for side walls
    bc_noslip_walls = dirichletbc(u_noslip, wall_dofs, V)

    #Combine top wall and side wall DoFs into a single boundary condition
    all_wall_dofs = np.array(np.concatenate((top_wall_dofs, wall_dofs)))
    #Create a single boundary condition for all walls
    bc_all = dirichletbc(u_noslip, all_wall_dofs, V)

    #Combine all boundary conditions into lists for assembly
    bcu = [bc_all]
    bcp = [bc_inflow, bc_outflow]

    #INITIAL VALUES AND PHYSICAL CONSTANTS
    #previous velocity
    u_n = Function(V)
    u_n.name = "u_n"

    #average velocity
    U = 0.5 * (u_n + u)
    n = FacetNormal(mesh)
    f = Constant(mesh, PETSc.ScalarType((0, 0, 0)))
    k = Constant(mesh, PETSc.ScalarType(dt))
    mu = Constant(mesh, PETSc.ScalarType(1))
    rho = Constant(mesh, PETSc.ScalarType(1))

    #Define strain-rate tensor
    def epsilon(u):
        return sym(nabla_grad(u))

    #Define stress tensor
    def sigma(u, p):
        return 2 * mu * epsilon(u) - p * Identity(len(u))

    #previous pressure
    p_n = Function(Q)
    p_n.name = "p_n"

    #STEP 1: Tentative velocity step
    F1 = rho * dot((u - u_n) / k, v) * dx
    F1 += rho * dot(dot(u_n, nabla_grad(u_n)), v) * dx
    F1 += inner(sigma(U, p_n), epsilon(v)) * dx
    F1 += dot(p_n * n, v) * ds - dot(mu * nabla_grad(U) * n, v) * ds
    F1 -= dot(f, v) * dx
    a1 = form(lhs(F1))
    L1 = form(rhs(F1))
    A1 = assemble_matrix(a1, bcs=bcu)
    A1.assemble()
    b1 = create_vector(L1)

    #STEP 2: Pressure correction
    u_ = Function(V)
    a2 = form(dot(nabla_grad(p), nabla_grad(q)) * dx)
    L2 = form(dot(nabla_grad(p_n), nabla_grad(q)) * dx - (rho / k) * div(u_) * q * dx)
    A2 = assemble_matrix(a2, bcs=bcp)
    A2.assemble()
    b2 = create_vector(L2)

    #STEP 3: Velocity correction
    p_ = Function(Q)
    a3 = form(rho * dot(u, v) * dx)
    L3 = form(rho * dot(u_, v) * dx - k * dot(nabla_grad(p_ - p_n), v) * dx)
    A3 = assemble_matrix(a3)
    A3.assemble()
    b3 = create_vector(L3)

    #Solver for step 1
    solver1 = PETSc.KSP().create(mesh.comm)
    solver1.setOperators(A1)
    solver1.setType(PETSc.KSP.Type.BCGS)
    pc1 = solver1.getPC()
    pc1.setType(PETSc.PC.Type.HYPRE)
    pc1.setHYPREType("boomeramg")

    #Solver for step 2
    solver2 = PETSc.KSP().create(mesh.comm)
    solver2.setOperators(A2)
    solver2.setType(PETSc.KSP.Type.BCGS)
    pc2 = solver2.getPC()
    pc2.setType(PETSc.PC.Type.HYPRE)
    pc2.setHYPREType("boomeramg")

    #Solver for step 3
    solver3 = PETSc.KSP().create(mesh.comm)
    solver3.setOperators(A3)
    solver3.setType(PETSc.KSP.Type.CG)
    pc3 = solver3.getPC()
    pc3.setType(PETSc.PC.Type.SOR)

    #TIME STEPPING LOOP
    from pathlib import Path
    folder = Path("results")
    folder.mkdir(exist_ok=True, parents=True)
    vtx_u = VTXWriter(mesh.comm, folder / "poiseuille_u.bp", u_n, engine="BP4")
    vtx_p = VTXWriter(mesh.comm, folder / "poiseuille_p.bp", p_n, engine="BP4")
    vtx_u.write(t)
    vtx_p.write(t)

    for i in range(num_steps):
        #Update current time step
        t += dt

        #Step 1: Tentative velocity step
        with b1.localForm() as loc_1:
            loc_1.set(0)
        assemble_vector(b1, L1)
        apply_lifting(b1, [a1], [bcu])
        b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        set_bc(b1, bcu)
        solver1.solve(b1, u_.x.petsc_vec)
        u_.x.scatter_forward()

        #Step 2: Pressure correction step
        with b2.localForm() as loc_2:
            loc_2.set(0)
        assemble_vector(b2, L2)
        apply_lifting(b2, [a2], [bcp])
        b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        set_bc(b2, bcp)
        solver2.solve(b2, p_.x.petsc_vec)
        p_.x.scatter_forward()

        #Step 3: Velocity correction step
        with b3.localForm() as loc_3:
            loc_3.set(0)
        assemble_vector(b3, L3)
        b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        solver3.solve(b3, u_.x.petsc_vec)
        u_.x.scatter_forward()

        #Update variable with solution form this time step
        u_n.x.array[:] = u_.x.array[:]
        p_n.x.array[:] = p_.x.array[:]


    #Close xmdf file
    vtx_u.close()
    vtx_p.close()
    


    #VISUALIZATION WITH PYVISTA
    pyvista.start_xvfb()
    topology, cell_types, geometry = vtk_mesh(V)
    values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
    values[:, :len(u_n)] = u_n.x.array.real.reshape((geometry.shape[0], len(u_n)))



    #Create a point cloud of glyphs
    function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    function_grid["u"] = values
    glyphs = function_grid.glyph(orient="u")#, factor=1)

    #Create a pyvista-grid for the mesh
    mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(*vtk_mesh(mesh, mesh.topology.dim))


    return glyphs, grid, V, Q, u_n, p_n

In [4]:
### Function to plot velocity profile along the y-axis at a fixed x and z
###
###


import matplotlib.pyplot as plt
import numpy as np

def plot_velocity_profile(V, u_n, mesh, x_target=4.0, tol=5e-1):
    """
    Plots the velocity profile along the y-axis 
    at x ≈ x_target and z ≈ the midpoint of the mesh.

    Parameters:
    - V: Function space of the solution (velocity space)
    - u_n: Solution (function containing velocity field)
    - mesh: Mesh object
    - x_target: x-coordinate of the slice
    - tol: Tolerance for coordinate proximity
    """
    # Extract coordinates of DoFs and velocity values
    coords = V.tabulate_dof_coordinates().reshape(-1, mesh.geometry.dim)
    u_n_values = u_n.x.array.reshape(-1, mesh.geometry.dim)

    # Target z-coordinate (midpoint of domain)
    z_target = round(mesh.geometry.x[:, 2].max() / 2, 1)

    # Filter points close to x_target and z_target
    mask = (np.isclose(coords[:, 0], x_target, atol=tol) &
            np.isclose(coords[:, 2], z_target, atol=tol))

    coords_filtered = coords[mask]
    u_filtered = u_n_values[mask]

    # Compute velocity magnitudes
    speeds = np.linalg.norm(u_filtered, axis=1)
    y_coords = coords_filtered[:, 1]

    # Round y-coordinates to group for averaging
    y_rounded = np.round(y_coords, decimals=3)
    unique_y = np.unique(y_rounded)
    avg_speeds = np.zeros(len(unique_y))

    for i, y_val in enumerate(unique_y):
        speed_vals = speeds[y_rounded == y_val]
        avg_speeds[i] = np.mean(speed_vals)

    # Plot velocity profile
    plt.figure(figsize=(6, 4))
    plt.plot(unique_y, avg_speeds, marker='o', label='Average speeds')
    plt.xlabel("y-coordinate")
    plt.ylabel("Mean velocity magnitude |u|")
    plt.title(f"Velocity profile at x ≈ {x_target}, z = {z_target}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()




In [5]:
### Function to plot pressure profile along the x-axis at a fixed y and z
###
###

import numpy as np
import matplotlib.pyplot as plt

def plot_pressure_profile(Q, p_n, mesh, y_target=4.0, tol=1e-1):
    """
    Plots the pressure profile along the x-axis
    at y ≈ y_target and z ≈ the midpoint of the mesh.

    Parameters:
    - Q: Function space of the pressure solution
    - p_n: Pressure solution function
    - mesh: Mesh object
    - y_target: y-coordinate of the slice
    - tol: Tolerance for coordinate proximity
    """
    # Extract coordinates and pressure values
    coords = Q.tabulate_dof_coordinates().reshape(-1, mesh.geometry.dim)
    p_n_values = p_n.x.array

    # Target z-coordinate (midpoint)
    z_target = round(mesh.geometry.x[:, 2].max() / 2, 1)

    # Filter points near y_target and z_target
    mask = (np.isclose(coords[:, 1], y_target, atol=tol) &
            np.isclose(coords[:, 2], z_target, atol=tol))

    coords_filtered = coords[mask]
    p_filtered = p_n_values[mask]

    # Extract x-coordinates of filtered points
    x_coords = coords_filtered[:, 0]

    # Group by rounded x-coordinates to average
    x_rounded = np.round(x_coords, decimals=3)
    unique_x = np.unique(x_rounded)
    avg_pressures = np.zeros(len(unique_x))

    for i, x_val in enumerate(unique_x):
        p_vals = p_filtered[x_rounded == x_val]
        avg_pressures[i] = np.mean(p_vals)

    # Plot pressure profile
    plt.figure(figsize=(6, 4))
    plt.plot(unique_x, avg_pressures, marker='o', label='Average Pressure')
    plt.xlabel("x-coordinate")
    plt.ylabel("Mean Pressure p")
    plt.title(f"Pressure profile at y ≈ {y_target}, z = {z_target}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
import yaml
def main():
    #Load configuration from YAML file
    with open("config.yaml") as f:
        config = yaml.safe_load(f)

    #Create meshes
    mesh3d, mesh2d = create_mesh_3d_and_2d(
        config["mesh"]["L"],
        config["mesh"]["W"],
        config["mesh"]["H"],
        config["mesh"]["nx"],
        config["mesh"]["ny"],
        config["mesh"]["nz"]
    )
    #Define pressure function
    pressure = pressure_function(mesh2d, config["pressure"])

    #Calculate deformation of the 2D membrane under pressure
    uh = calculate_deformation(mesh2d, pressure_function(mesh2d, pressure))

    ##Deform the 3D mesh based on the 2D membrane solution
    #membrane_deformation = plot_membrane_deformation(
    #    mesh2d,
    #    mesh3d,
    #    pressure,
    #    calculate_deformation
    #)
    deformed_mesh = mesh_deformation(uh.x.array, mesh3d)

    #Solve the Navier-Stokes equations on the deformed mesh
    glyphs, grid, V, Q, u_n, p_n = solve_navier_stokes(
        deformed_mesh,
        t=config["simulation"]["t"],
        T=config["simulation"]["T"],
        num_steps=config["simulation"]["num_steps"],
        p_in=config["simulation"]["pressure"]["inflow"],
        p_out=config["simulation"]["pressure"]["outflow"]
    )

    ##Plot velocity and pressure profiles
    #velocity_plot = plot_velocity_profile(
    #    V, u_n, deformed_mesh,
    #    x_target=config["simulation"]["velocity_plot"]["x_target"],
    #    tol=config["simulation"]["velocity_plot"]["tol"]
    #)
    
    #pressure_plot = plot_pressure_profile(
    #    Q, p_n, deformed_mesh,
    #    y_target=config["simulation"]["pressure_plot"]["y_target"],
    #    tol=config["simulation"]["pressure_plot"]["tol"]
    #)

    
# Nur ausführen, wenn direkt gestartet (nicht importiert)
if __name__ == "__main__":
    main()